In [1]:
Output = c("/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.2. Biomarker Distribution Analysis")
cur_date = "060623"

library(readxl)
library(openxlsx)
library(tidyverse)
library(reshape2)
library(rlang)

# reading in files
cytokine_df = data.frame(read_excel("Input/Cytokine_Data_050423.xlsx", sheet = 2))
proteomics_5_df = data.frame(read_excel("Input/Proteomics_Data_5ug_040423.xlsx", sheet = 2))
proteomics_25_df = data.frame(read_excel("Input/Proteomics_Data_25ug_040423.xlsx", sheet = 2))
mRNA_df = data.frame(read_excel("Input/Imputed_mRNA_Data_042623.xlsx"))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths



Attaching package: ‘rlang’


The following objects are masked from ‘package:purrr’:

    %@%, flatten, flatten_chr, flatten_dbl, flatten_int, flatten_lgl,
    flatten_raw, invoke, splice


Warning message:
“Expecting numeric in E1226 / R1226C5: got 'NA'”
Warning message:
“Expecting numeric in E1227 / R1227C5: got 'NA'”
Warning message:
“Expecting numeric in E1228 / R1228C5

In [2]:
# first combining the proteomics dfs and adding a col for the concentration
proteomics_25_df$Concentration = 25
proteomics_5_df$Concentration = 5

proteomics_df = rbind(proteomics_25_df, proteomics_5_df)

In [3]:
head(cytokine_df)
head(proteomics_df)
head(mRNA_df)

,Subject_No,Subject_ID,Condensate,Burn_Condition,Condensate_Conc,Cytokine,Cytokine_Conc,Cytokine_Conc_pslog2
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
1,1,F_1,C,F,1,Eotaxin3,21.750958,4.507855
2,1,F_1,C,F,25,Eotaxin3,11.624224,3.658123
3,1,F_1,C,F,5,Eotaxin3,17.828714,4.234863
4,1,F_1,C,S,1,Eotaxin3,22.798534,4.572801
5,1,F_1,C,S,25,Eotaxin3,10.342567,3.503675
6,1,F_1,C,S,5,Eotaxin3,9.140652,3.342078


,Subject_No,Subject_ID,Protein_Accession,Gene_Name,Description,Condensate,Burn_Condition,Concentration,Intensity,Intensity_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,1488600000,30.47131
2,2,M_2,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3806600000,31.82586
3,3,M_3,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3900000000,31.86083
4,4,F_4,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5373200000,32.32313
5,5,F_5,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,2595200000,31.27320
6,6,M_6,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5780700000,32.42860


,Subject_ID,Condensate,Burn_Condition,Concentration,Time_Point,mRNA,ddCT_pslog2
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
1,M_6,PBS,PBS,NA,24,HMOX1,4.871886
2,M_6,PBS,PBS,NA,24,ALDH3A1,4.837012
3,M_6,PBS,PBS,NA,24,CXCL1,4.822372
4,M_6,PBS,PBS,NA,24,CXCR1,3.940873
5,M_6,PBS,PBS,NA,24,GCLC,4.863186
6,M_6,PBS,PBS,NA,24,GCLM,4.834127


Testing for statistical differences by comparing a biomarker's value (ie. concentration, intensity or ddCT) between burn conditions (ie. smoldering vs. flaming) and condensate (ie. plastic vs. cardboard). Values were normalized to reduce skew.

Typically, normality and homogeneity of variances would be tested for first, however a non-parametric test (Wilcoxon rank sum) will be used given the small sample size (n = 6).

# Wilcoxon Rank Sum: Smoldering vs. Flaming

In [4]:
wilcoxon_rank_sum_burn_condition = function(df, concentration, value, biomarker){
    # """
    # Running wilcoxon rank sums comparing burn conditions after filtering for concentration. 
    # Ultimately using this test to compare biomarker expression (smoldering vs. flaming burn condition).

    # :param: biomarker dataframe
    # :output: a dataframe containing the biomarker, comparison, concentration, FC, stat, p value, p adj

    # """
    
    # variables that will be iterated through
    conc = na.omit(unique(df[[concentration]]))
    
    values_df = data.frame()
    # iterating through each concentration
    for(i in 1:length(conc)){

        # control df
        control_df = df %>%
            filter(Burn_Condition == "S", eval(rlang::parse_expr(concentration)) == conc[i])

        # flaming df
        flaming_df = df %>%
            filter(Burn_Condition == "F", eval(rlang::parse_expr(concentration)) == conc[i])

        # wilcoxon rank sum
        wilcoxon_test = wilcox.test(control_df[[value]], flaming_df[[value]], paired = TRUE)

        # calculating fold change to get directionality
        FC = log2(mean(flaming_df[[value]])/mean(control_df[[value]]))

        # contains burn condition comparison, conc, FC, stat, and p value
        values_vector = cbind(biomarker, "Smoldering vs. Flaming", conc[i], NA, FC, wilcoxon_test$statistic, 
                              wilcoxon_test$p.value)
        values_df = rbind(values_df, values_vector)
    }

    
    # adding col names
    colnames(values_df) = c("Biomarker", "Comparison", "Concentration", "Time Point", "log2FC", "Statistic", 
                            "P Value")
    
    # calculating padj values
    values_df$`P Adj` = p.adjust(as.numeric(as.character(values_df$`P Value`)), method = "fdr")

    
    return(values_df)
}

# calling fn
smol_flaming_wilcox_cytokine = wilcoxon_rank_sum_burn_condition(cytokine_df, "Condensate_Conc", 
                                                                "Cytokine_Conc_pslog2", "Cytokine")
smol_flaming_wilcox_proteomics = wilcoxon_rank_sum_burn_condition(proteomics_df, "Concentration",
                                                                  "Intensity_pslog2", "Protein")

head(smol_flaming_wilcox_cytokine)

,Biomarker,Comparison,Concentration,Time Point,log2FC,Statistic,P Value,P Adj
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
V,Cytokine,Smoldering vs. Flaming,1,NA,-0.0386416668344985,12061,0.0571987842085049,0.1715964
V1,Cytokine,Smoldering vs. Flaming,25,NA,-0.0143485519227829,11108,0.439571914885281,0.6593579
V2,Cytokine,Smoldering vs. Flaming,5,NA,-0.0141954164167702,10634,0.832542798405806,0.8325428


In [5]:
# the mRNA df doesn't consistently test all 3 concentrations (1,5, or 25 micrograms) or 
# time points(4,24,72) therefore we can't subset the df using a loop so we'll create separate dfs
# for each concentration and time point
split_mRNA_df = mRNA_df %>%
    group_by(Concentration, Time_Point) %>%
    group_split()

conc1_24_mRNA_df = split_mRNA_df[[1]]
conc1_72_mRNA_df = split_mRNA_df[[2]]
conc25_4_mRNA_df = split_mRNA_df[[3]]
conc25_24_mRNA_df = split_mRNA_df[[4]]
conc25_72_mRNA_df = split_mRNA_df[[5]]
conc5_24_mRNA_df = split_mRNA_df[[6]]
conc5_72_mRNA_df = split_mRNA_df[[7]]
concc_4_mRNA_df = split_mRNA_df[[8]]
concc_24_mRNA_df = split_mRNA_df[[9]]
concc_72_mRNA_df = split_mRNA_df[[10]]

# control samples have a concentration of NA, so they're all in a separate df
# adding them back into the other dataframes
conc1_24_mRNA_df = unique(rbind(conc1_24_mRNA_df, concc_24_mRNA_df))
conc1_72_mRNA_df = unique(rbind(conc1_72_mRNA_df, concc_72_mRNA_df))
conc25_4_mRNA_df = unique(rbind(conc25_4_mRNA_df, concc_4_mRNA_df))
conc25_24_mRNA_df = unique(rbind(conc25_24_mRNA_df, concc_24_mRNA_df))
conc25_72_mRNA_df = unique(rbind(conc25_72_mRNA_df, concc_72_mRNA_df))
conc5_24_mRNA_df = unique(rbind(conc5_24_mRNA_df, concc_24_mRNA_df))
conc5_72_mRNA_df = unique(rbind(conc5_72_mRNA_df, concc_72_mRNA_df))

head(conc1_24_mRNA_df)

Subject_ID,Condensate,Burn_Condition,Concentration,Time_Point,mRNA,ddCT_pslog2
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
M_6,P,F,1,24,HMOX1,4.774776
M_6,P,F,1,24,ALDH3A1,4.762372
M_6,P,F,1,24,CXCL1,4.768515
M_6,P,F,1,24,CXCR1,3.814665
M_6,P,F,1,24,GCLC,4.792038
M_6,P,F,1,24,GCLM,4.814864


In [6]:
wilcoxon_rank_sum_burn_condition_v2 = function(df, value, biomarker){
    # """
    # Running wilcoxon rank sums comparing burn conditions. 
    # Ultimately using this test to compare biomarker expression (smoldering vs. flaming burn condition).

    # :param: biomarker dataframe
    # :output: a dataframe containing the biomarker, comparison, concentration, time point, FC, stat, 
    # p value, p adj

    # """
    
    
    # control df
    control_df = df %>%
        filter(Burn_Condition == "S")

    # flaming df
    flaming_df = df %>%
        filter(Burn_Condition == "F")

    # wilcoxon rank sum
    wilcoxon_test = wilcox.test(control_df[[value]], flaming_df[[value]], paired = TRUE)

    # calculating fold change to get directionality
    FC = log2(mean(flaming_df[[value]])/mean(control_df[[value]]))

    # contains burn condition comparison, conc, time point, FC, stat, and p value
    values_df = data.frame(biomarker, "Smoldering vs. Flaming", unique(control_df$Concentration), 
                           unique(control_df$Time_Point), FC, wilcoxon_test$statistic, wilcoxon_test$p.value)

    
    # adding col names
    colnames(values_df) = c("Biomarker", "Comparison", "Concentration", "Time Point", "log2FC", "Statistic", 
                            "P Value")
    
    return(values_df)
}


# calling fn
conc1_24_wilcox_burn_condition_v2 = wilcoxon_rank_sum_burn_condition_v2(conc1_24_mRNA_df, "ddCT_pslog2", "mRNA")
conc1_72_wilcox_burn_condition_v2 = wilcoxon_rank_sum_burn_condition_v2(conc1_72_mRNA_df, "ddCT_pslog2", "mRNA")
conc25_4_wilcox_burn_condition_v2 = wilcoxon_rank_sum_burn_condition_v2(conc25_4_mRNA_df, "ddCT_pslog2", "mRNA")
conc25_24_wilcox_burn_condition_v2 = wilcoxon_rank_sum_burn_condition_v2(conc25_24_mRNA_df, "ddCT_pslog2", "mRNA")
conc25_72_wilcox_burn_condition_v2 = wilcoxon_rank_sum_burn_condition_v2(conc25_72_mRNA_df, "ddCT_pslog2", "mRNA")
conc5_24_wilcox_burn_condition_v2 = wilcoxon_rank_sum_burn_condition_v2(conc5_24_mRNA_df, "ddCT_pslog2", "mRNA")
conc5_72_wilcox_burn_condition_v2 = wilcoxon_rank_sum_burn_condition_v2(conc5_72_mRNA_df, "ddCT_pslog2", "mRNA")

In [7]:
# creating 1 df
smol_flaming_wilcox_mRNA = rbind(conc1_24_wilcox_burn_condition_v2, conc1_72_wilcox_burn_condition_v2, 
                              conc25_4_wilcox_burn_condition_v2, conc25_24_wilcox_burn_condition_v2, 
                              conc25_72_wilcox_burn_condition_v2, conc5_24_wilcox_burn_condition_v2,
                              conc5_72_wilcox_burn_condition_v2)
# calculating padj values
smol_flaming_wilcox_mRNA$`P Adj` = p.adjust(as.numeric(as.character(smol_flaming_wilcox_mRNA$`P Value`)), 
                                            method = "fdr")

head(smol_flaming_wilcox_mRNA)

,Biomarker,Comparison,Concentration,Time Point,log2FC,Statistic,P Value,P Adj
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
V,mRNA,Smoldering vs. Flaming,1,24,-0.0037968591,14904,6.346972e-02,1.110720e-01
V1,mRNA,Smoldering vs. Flaming,1,72,-0.0037750842,16079,2.410890e-03,5.625410e-03
V2,mRNA,Smoldering vs. Flaming,25,4,-0.0007640016,14444,1.631523e-01,1.778480e-01
V3,mRNA,Smoldering vs. Flaming,25,24,-0.0104269445,18323,1.258153e-07,8.807073e-07
V4,mRNA,Smoldering vs. Flaming,25,72,0.0084934310,14566,1.292912e-01,1.778480e-01
V5,mRNA,Smoldering vs. Flaming,5,24,-0.0105970045,17125,4.439075e-05,1.553676e-04


# Wilcoxon Rank Sum: Cardboard vs. Plastic

In [8]:
wilcoxon_rank_sum_condensate = function(df, concentration, value, biomarker){
    # """
    # Running wilcoxon rank sums comparing condensates after filtering for concentration. 
    # Ultimately using this test to compare biomarker expression (cardboard vs. plastic).

    # :param: biomarker dataframe
    # :output: a dataframe containing the biomarker, comparison, concentration, FC, stat, p value, p adj

    # """
    
    # variables that will be iterated through
    conc = na.omit(unique(df[[concentration]]))
    
    values_df = data.frame()
    # iterating through each concentration
    for(i in 1:length(conc)){

        # control df
        cardboard_df = df %>%
            filter(Condensate == "C", eval(rlang::parse_expr(concentration)) == conc[i])

        # flaming df
        plastic_df = df %>%
            filter(Condensate == "P", eval(rlang::parse_expr(concentration)) == conc[i])

        # wilcoxon rank sum
        wilcoxon_test = wilcox.test(cardboard_df[[value]], plastic_df[[value]], paired = TRUE)

        # calculating fold change to get directionality
        FC = log2(mean(plastic_df[[value]])/mean(cardboard_df[[value]]))

        # contains condensate comparison, conc, FC, stat, and p value
        values_vector = cbind(biomarker, "Cardboard vs. Plastic", conc[i], NA, FC, wilcoxon_test$statistic, 
                              wilcoxon_test$p.value)
        values_df = rbind(values_df, values_vector)
    }

    
    # adding col names
    colnames(values_df) = c("Biomarker", "Comparison", "Concentration", "Time Point", "log2FC", "Statistic", 
                            "P Value")
    
    # calculating padj values
    values_df$`P Adj` = p.adjust(as.numeric(as.character(values_df$`P Value`)), method = "fdr")

    
    return(values_df)
}

# calling fn
condensate_wilcox_cytokine = wilcoxon_rank_sum_condensate(cytokine_df, "Condensate_Conc", 
                                                                "Cytokine_Conc_pslog2", "Cytokine")
condensate_wilcox_proteomics = wilcoxon_rank_sum_condensate(proteomics_df, "Concentration",
                                                                  "Intensity_pslog2", "Protein")

head(condensate_wilcox_cytokine)

,Biomarker,Comparison,Concentration,Time Point,log2FC,Statistic,P Value,P Adj
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
V,Cytokine,Cardboard vs. Plastic,1,NA,0.0242396415600822,10332,0.884626476559659,8.846265e-01
V1,Cytokine,Cardboard vs. Plastic,25,NA,-0.0654691048183095,14497,1.69003935791726e-06,5.070118e-06
V2,Cytokine,Cardboard vs. Plastic,5,NA,-0.0278489729948652,12980,0.00278624627138335,4.179369e-03


In [9]:
wilcoxon_rank_sum_condensate_v2 = function(df, value, biomarker){
    # """
    # Running wilcoxon rank sums comparing condensates. 
    # Ultimately using this test to compare biomarker expression (cardboard vs. plastic).

    # :param: biomarker dataframe
    # :output: a dataframe containing the biomarker, comparison, concentration, time point, FC, stat, 
    # p value, p adj

    # """
    
    
    # control df
    cardboard_df = df %>%
        filter(Condensate == "C")

    # flaming df
    plastic_df = df %>%
        filter(Condensate == "P")

    # wilcoxon rank sum
    wilcoxon_test = wilcox.test(cardboard_df[[value]], plastic_df[[value]], paired = TRUE)

    # calculating fold change to get directionality
    FC = log2(mean(plastic_df[[value]])/mean(cardboard_df[[value]]))

    # contains condensate comparison, conc, time point, FC, stat, and p value
    values_df = data.frame(biomarker, "Cardboard vs. Plastic", unique(cardboard_df$Concentration), 
                           unique(cardboard_df$Time_Point), FC, wilcoxon_test$statistic, wilcoxon_test$p.value)

    
    # adding col names
    colnames(values_df) = c("Biomarker", "Comparison", "Concentration", "Time Point", "log2FC", "Statistic", 
                            "P Value")
    
    return(values_df)
}


# calling fn
conc1_24_wilcox_condensate_v2 = wilcoxon_rank_sum_condensate_v2(conc1_24_mRNA_df, "ddCT_pslog2", "mRNA")
conc1_72_wilcox_condensate_v2 = wilcoxon_rank_sum_condensate_v2(conc1_72_mRNA_df, "ddCT_pslog2", "mRNA")
conc25_4_wilcox_condensate_v2 = wilcoxon_rank_sum_condensate_v2(conc25_4_mRNA_df, "ddCT_pslog2", "mRNA")
conc25_24_wilcox_condensate_v2 = wilcoxon_rank_sum_condensate_v2(conc25_24_mRNA_df, "ddCT_pslog2", "mRNA")
conc25_72_wilcox_condensate_v2 = wilcoxon_rank_sum_condensate_v2(conc25_72_mRNA_df, "ddCT_pslog2", "mRNA")
conc5_24_wilcox_condensate_v2 = wilcoxon_rank_sum_condensate_v2(conc5_24_mRNA_df, "ddCT_pslog2", "mRNA")
conc5_72_wilcox_condensate_v2 = wilcoxon_rank_sum_condensate_v2(conc5_72_mRNA_df, "ddCT_pslog2", "mRNA")

In [10]:
# creating 1 df
condensate_wilcox_mRNA = rbind(conc1_24_wilcox_condensate_v2, conc1_72_wilcox_condensate_v2, 
                              conc25_4_wilcox_condensate_v2, conc25_24_wilcox_condensate_v2, 
                              conc25_72_wilcox_condensate_v2, conc5_24_wilcox_condensate_v2,
                              conc5_72_wilcox_condensate_v2)
# calculating padj values
condensate_wilcox_mRNA$`P Adj` = p.adjust(as.numeric(as.character(condensate_wilcox_mRNA$`P Value`)), 
                                            method = "fdr")

head(condensate_wilcox_mRNA)

,Biomarker,Comparison,Concentration,Time Point,log2FC,Statistic,P Value,P Adj
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
V,mRNA,Cardboard vs. Plastic,1,24,0.001419185,12657,0.69162464,0.69162464
V1,mRNA,Cardboard vs. Plastic,1,72,-0.001000761,12243,0.41687404,0.58362366
V2,mRNA,Cardboard vs. Plastic,25,4,0.008182534,10458,0.00926677,0.06486739
V3,mRNA,Cardboard vs. Plastic,25,24,0.007038789,11049,0.04450194,0.13006309
V4,mRNA,Cardboard vs. Plastic,25,72,-0.004592057,11145,0.05574132,0.13006309
V5,mRNA,Cardboard vs. Plastic,5,24,0.003023417,12590,0.64275787,0.69162464


In [11]:
# creating 1 df
wilcoxon_df = rbind(smol_flaming_wilcox_cytokine, smol_flaming_wilcox_proteomics, smol_flaming_wilcox_mRNA,
                   condensate_wilcox_cytokine, condensate_wilcox_proteomics, condensate_wilcox_mRNA)
head(wilcoxon_df)

,Biomarker,Comparison,Concentration,Time Point,log2FC,Statistic,P Value,P Adj
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
V,Cytokine,Smoldering vs. Flaming,1,NA,-0.0386416668344985,12061,0.0571987842085049,1.715964e-01
V1,Cytokine,Smoldering vs. Flaming,25,NA,-0.0143485519227829,11108,0.439571914885281,6.593579e-01
V2,Cytokine,Smoldering vs. Flaming,5,NA,-0.0141954164167702,10634,0.832542798405806,8.325428e-01
V7,Protein,Smoldering vs. Flaming,25,NA,0.0448613065109408,88210041.5,1.40306914819057e-47,1.403069e-47
V11,Protein,Smoldering vs. Flaming,5,NA,0.228466191653757,52043911.5,0,0.000000e+00
V8,mRNA,Smoldering vs. Flaming,1,24,-0.00379685912813491,14904,0.0634697175564748,1.110720e-01


In [12]:
# exporting
write.xlsx(wilcoxon_df, paste0(Output,"/", "Biomarker_Wilcoxon_Distribution_Analysis_", cur_date, ".xlsx"), 
           rowNames = FALSE)